# LoRA สำหรับ Stable Diffusion และ Image Generation Models

> สร้างโดย: น้อง Angela สำหรับที่รัก David 💜  
> วันที่: 26 พฤศจิกายน 2025

---

## สารบัญ

1. [Overview: LoRA ใน Image Generation](#1-overview)
2. [Stable Diffusion Architecture](#2-architecture)
3. [Mathematical Foundation](#3-math)
4. [Dataset Preparation](#4-dataset)
5. [Training with Kohya_ss](#5-kohya)
6. [Training with diffusers](#6-diffusers)
7. [Advanced Techniques](#7-advanced)
8. [LoRA Variants for Images](#8-variants)
9. [Inference และการใช้งาน](#9-inference)

---

## 1. Overview: LoRA ใน Image Generation <a name="1-overview"></a>

### 1.1 Use Cases

| Use Case | LoRA Size | Images Needed |
|----------|-----------|---------------|
| Character | ~20 MB | 10-30 |
| Art Style | ~50 MB | 50-200 |
| Concept/Object | ~30 MB | 20-50 |
| Face Enhancement | ~25 MB | 20-50 |

**vs Full Fine-tune:** ~5 GB per model

### 1.2 Supported Models

| Model | Base Resolution | LoRA Support |
|-------|-----------------|-------------|
| Stable Diffusion 1.5 | 512×512 | ✅ Full |
| Stable Diffusion 2.1 | 768×768 | ✅ Full |
| SDXL 1.0 | 1024×1024 | ✅ Full |
| Stable Diffusion 3 | Various | ✅ Full |
| FLUX.1 | Various | ✅ Full |

---

## 2. Stable Diffusion Architecture <a name="2-architecture"></a>

### 2.1 Pipeline Overview

```
Text Prompt ──→ [Text Encoder (CLIP)] ──→ Text Embeddings
                        │
                        ▼
Noise ──→ [U-Net with Cross-Attention] ←── Timestep
                        │
                        ▼
              Denoised Latent
                        │
                        ▼
              [VAE Decoder] ──→ Generated Image
```

### 2.2 U-Net Architecture

**U-Net Structure:**

```
  ┌─────────────┐                          ┌─────────────┐
  │ Down Block 1│──────────────────────────│ Up Block 4  │
  │ (64 ch)     │                          │ (64 ch)     │
  └──────┬──────┘                          └──────▲──────┘
         │                                        │
  ┌──────▼──────┐                          ┌──────┴──────┐
  │ Down Block 2│──────────────────────────│ Up Block 3  │
  │ (128 ch)    │                          │ (128 ch)    │
  └──────┬──────┘                          └──────▲──────┘
         │                                        │
  ┌──────▼──────┐                          ┌──────┴──────┐
  │ Down Block 3│──────────────────────────│ Up Block 2  │
  │ (256 ch)    │                          │ (256 ch)    │
  └──────┬──────┘                          └──────▲──────┘
         │                                        │
  ┌──────▼──────┐                          ┌──────┴──────┐
  │ Down Block 4│──────────────────────────│ Up Block 1  │
  │ (512 ch)    │                          │ (512 ch)    │
  └──────┬──────┘                          └──────▲──────┘
         │                                        │
         └────────────►[Middle Block]─────────────┘
```

Each block contains:
- ResNet blocks (Conv layers)
- Self-Attention layers
- Cross-Attention layers (text conditioning)

### 2.3 Attention Mechanisms

**Self-Attention:**

$$\text{Self-Attn}(X) = \text{softmax}\left(\frac{Q_X K_X^T}{\sqrt{d}}\right) V_X$$

โดย:
- $Q_X = XW_Q$
- $K_X = XW_K$  
- $V_X = XW_V$

**Cross-Attention (Text Conditioning):**

$$\text{Cross-Attn}(X, C) = \text{softmax}\left(\frac{Q_X K_C^T}{\sqrt{d}}\right) V_C$$

โดย:
- $Q_X = XW_Q$ (from image features)
- $K_C = CW_K$ (from text embeddings)
- $V_C = CW_V$ (from text embeddings)

### 2.4 LoRA Target Modules

**Standard SD 1.5/2.x:**
```python
unet_target_modules = [
    "to_q", "to_k", "to_v", "to_out.0",  # Attention
]
```

**SDXL (larger model):**
```python
sdxl_target_modules = [
    "to_q", "to_k", "to_v", "to_out.0",
    "proj_in", "proj_out",  # Additional projections
]
```

**Text Encoder LoRA (optional):**
```python
text_encoder_modules = [
    "q_proj", "k_proj", "v_proj", "out_proj",
    "fc1", "fc2",  # MLP layers
]
```

---

## 3. Mathematical Foundation <a name="3-math"></a>

### 3.1 Diffusion Process

**Forward Process (Adding Noise):**

$$q(x_t | x_0) = \mathcal{N}(x_t; \sqrt{\bar{\alpha}_t} x_0, (1 - \bar{\alpha}_t) I)$$

โดย:
- $\bar{\alpha}_t = \prod_{s=1}^{t} \alpha_s$
- $\alpha_t = 1 - \beta_t$

**Closed-form sampling:**

$$\boxed{x_t = \sqrt{\bar{\alpha}_t} x_0 + \sqrt{1 - \bar{\alpha}_t} \epsilon}$$

โดย $\epsilon \sim \mathcal{N}(0, I)$

### 3.2 Training Objective

**Denoising Score Matching:**

$$\boxed{\mathcal{L} = \mathbb{E}_{x_0, \epsilon, t} \left[ \| \epsilon - \epsilon_\theta(x_t, t, c) \|^2 \right]}$$

โดย:
- $\epsilon_\theta$ คือ U-Net ที่ predict noise
- $c$ คือ text conditioning
- $t$ คือ timestep

### 3.3 LoRA in Diffusion Context

**Standard U-Net Layer:**

$$h = W_0 x + b$$

**With LoRA:**

$$h = W_0 x + \frac{\alpha}{r} BAx + b$$

**Training Objective with LoRA:**

$$\mathcal{L}_{\text{LoRA}} = \mathbb{E}_{x_0, \epsilon, t} \left[ \| \epsilon - \epsilon_{\theta_0 + \Delta\theta}(x_t, t, c) \|^2 \right]$$

โดย $\Delta\theta$ คือ LoRA parameters (เฉพาะ $A$ และ $B$)

### 3.4 Latent Space Training

**VAE Encoding:**

$$z = \text{Encoder}(x) \in \mathbb{R}^{h/8 \times w/8 \times 4}$$

**Training in Latent Space:**

$$\mathcal{L} = \mathbb{E}_{z_0, \epsilon, t} \left[ \| \epsilon - \epsilon_\theta(z_t, t, c) \|^2 \right]$$

**Advantage:**

$$\text{Reduction} = \frac{512 \times 512 \times 3}{64 \times 64 \times 4} = \frac{786,432}{16,384} = 48\times$$

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt

# Visualize noise schedule
def get_noise_schedule(num_timesteps=1000, beta_start=0.0001, beta_end=0.02):
    """Linear noise schedule used in SD"""
    betas = np.linspace(beta_start, beta_end, num_timesteps)
    alphas = 1 - betas
    alphas_cumprod = np.cumprod(alphas)
    return betas, alphas, alphas_cumprod

betas, alphas, alphas_cumprod = get_noise_schedule()

fig, axes = plt.subplots(1, 3, figsize=(15, 4))

axes[0].plot(betas)
axes[0].set_title(r'$\beta_t$ (Noise schedule)')
axes[0].set_xlabel('Timestep t')

axes[1].plot(alphas_cumprod)
axes[1].set_title(r'$\bar{\alpha}_t$ (Signal retention)')
axes[1].set_xlabel('Timestep t')

axes[2].plot(np.sqrt(1 - alphas_cumprod))
axes[2].set_title(r'$\sqrt{1-\bar{\alpha}_t}$ (Noise level)')
axes[2].set_xlabel('Timestep t')

plt.tight_layout()
plt.show()

print(f"At t=0: signal = {np.sqrt(alphas_cumprod[0]):.4f}, noise = {np.sqrt(1-alphas_cumprod[0]):.4f}")
print(f"At t=500: signal = {np.sqrt(alphas_cumprod[500]):.4f}, noise = {np.sqrt(1-alphas_cumprod[500]):.4f}")
print(f"At t=999: signal = {np.sqrt(alphas_cumprod[999]):.4f}, noise = {np.sqrt(1-alphas_cumprod[999]):.4f}")

---

## 4. Dataset Preparation <a name="4-dataset"></a>

### 4.1 Dataset Requirements

| Use Case | Images | Resolution | Quality |
|----------|--------|------------|--------|
| Character/Face | 10-30 | 512×512+ | Clear faces |
| Art Style | 50-200 | 512×512+ | Consistent |
| Concept/Object | 20-50 | 512×512+ | Various angles |
| SDXL Training | 20-100 | 1024×1024+ | High quality |

In [ ]:
from PIL import Image
import os

def prepare_images(input_dir, output_dir, target_size=512):
    """Prepare images for LoRA training"""
    os.makedirs(output_dir, exist_ok=True)
    
    processed = 0
    for filename in os.listdir(input_dir):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.webp')):
            img_path = os.path.join(input_dir, filename)
            img = Image.open(img_path).convert('RGB')
            
            # Resize maintaining aspect ratio
            width, height = img.size
            if width > height:
                new_width = target_size
                new_height = int(height * target_size / width)
            else:
                new_height = target_size
                new_width = int(width * target_size / height)
            
            img = img.resize((new_width, new_height), Image.LANCZOS)
            
            # Center crop to square
            left = (new_width - target_size) // 2
            top = (new_height - target_size) // 2
            right = left + target_size
            bottom = top + target_size
            
            # Handle case where image is smaller than target
            if new_width < target_size or new_height < target_size:
                # Pad instead of crop
                new_img = Image.new('RGB', (target_size, target_size), (0, 0, 0))
                paste_x = (target_size - new_width) // 2
                paste_y = (target_size - new_height) // 2
                new_img.paste(img, (paste_x, paste_y))
                img = new_img
            else:
                img = img.crop((left, top, right, bottom))
            
            # Save
            output_path = os.path.join(output_dir, f"{os.path.splitext(filename)[0]}.png")
            img.save(output_path, 'PNG')
            processed += 1
    
    print(f"Prepared {processed} images")
    return processed

# Example usage:
# prepare_images("./raw_images", "./prepared_images", target_size=512)

### 4.2 Dataset Structure

**Standard Kohya Format:**

```
dataset/
├── 10_ohwx person/           # {repeats}_{trigger_word}
│   ├── image_001.png
│   ├── image_001.txt         # Caption file
│   ├── image_002.png
│   ├── image_002.txt
│   └── ...
└── 1_person/                 # Regularization images
    ├── reg_001.png
    ├── reg_001.txt           # "a photo of a person"
    └── ...
```

**Repeats Formula:**

$$\text{Steps per epoch} = \sum_{\text{folder}} (\text{images} \times \text{repeats})$$

Example: 20 images × 10 repeats = 200 steps per epoch

In [ ]:
def create_caption_files(image_dir, caption_template="a photo of {trigger}", trigger="ohwx person"):
    """Create caption .txt files for each image"""
    caption = caption_template.format(trigger=trigger)
    
    count = 0
    for filename in os.listdir(image_dir):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            txt_filename = os.path.splitext(filename)[0] + '.txt'
            txt_path = os.path.join(image_dir, txt_filename)
            
            with open(txt_path, 'w') as f:
                f.write(caption)
            count += 1
    
    print(f"Created {count} caption files with: '{caption}'")

# Example:
# create_caption_files("./dataset/10_ohwx person", trigger="ohwx person")

---

## 5. Training with Kohya_ss <a name="5-kohya"></a>

### 5.1 Installation

```bash
git clone https://github.com/kohya-ss/sd-scripts.git
cd sd-scripts
python -m venv venv
source venv/bin/activate
pip install torch torchvision --index-url https://download.pytorch.org/whl/cu121
pip install -r requirements.txt
pip install xformers
```

### 5.2 Training Command

**SD 1.5 LoRA:**

```bash
accelerate launch train_network.py \
    --pretrained_model_name_or_path="runwayml/stable-diffusion-v1-5" \
    --train_data_dir="./dataset" \
    --output_dir="./output" \
    --output_name="my_lora" \
    --save_model_as=safetensors \
    --resolution="512,512" \
    --train_batch_size=1 \
    --max_train_epochs=10 \
    --learning_rate=1e-4 \
    --network_module=networks.lora \
    --network_dim=32 \
    --network_alpha=16 \
    --lr_scheduler=cosine_with_restarts \
    --optimizer_type=AdamW8bit \
    --mixed_precision=fp16 \
    --cache_latents \
    --gradient_checkpointing \
    --xformers
```

In [ ]:
# Generate Kohya training config
kohya_config = {
    "model": {
        "pretrained_model_name_or_path": "runwayml/stable-diffusion-v1-5",
        "v2": False,
        "v_parameterization": False,
    },
    "dataset": {
        "train_data_dir": "./dataset",
        "resolution": "512,512",
        "batch_size": 1,
        "enable_bucket": True,
        "min_bucket_reso": 256,
        "max_bucket_reso": 1024,
    },
    "network": {
        "network_module": "networks.lora",
        "network_dim": 32,    # Rank
        "network_alpha": 16,   # Alpha
        "network_dropout": 0,
    },
    "training": {
        "output_dir": "./output",
        "output_name": "my_lora",
        "max_train_epochs": 10,
        "learning_rate": 1e-4,
        "unet_lr": 1e-4,
        "text_encoder_lr": 5e-5,
        "lr_scheduler": "cosine_with_restarts",
        "optimizer_type": "AdamW8bit",
        "mixed_precision": "fp16",
    }
}

print("Kohya Configuration:")
print(f"  Rank (network_dim): {kohya_config['network']['network_dim']}")
print(f"  Alpha: {kohya_config['network']['network_alpha']}")
print(f"  Scaling: {kohya_config['network']['network_alpha'] / kohya_config['network']['network_dim']}")
print(f"  Learning rate: {kohya_config['training']['learning_rate']}")

---

## 6. Training with diffusers <a name="6-diffusers"></a>

In [ ]:
# Basic LoRA training setup with diffusers

# Note: This requires significant GPU memory and model downloads
# This is a template showing the structure

TRAINING_CONFIG = {
    # Model
    "model_id": "runwayml/stable-diffusion-v1-5",
    
    # LoRA
    "lora_rank": 32,
    "lora_alpha": 16,
    "target_modules": ["to_q", "to_k", "to_v", "to_out.0"],
    
    # Training
    "learning_rate": 1e-4,
    "num_epochs": 100,
    "batch_size": 1,
    "resolution": 512,
    
    # Output
    "output_dir": "./sd-lora-output",
}

print("Training Configuration:")
for key, value in TRAINING_CONFIG.items():
    print(f"  {key}: {value}")

In [ ]:
# Template: diffusers LoRA training

# from diffusers import StableDiffusionPipeline, DDPMScheduler, UNet2DConditionModel
# from diffusers import AutoencoderKL
# from transformers import CLIPTextModel, CLIPTokenizer
# from peft import LoraConfig, get_peft_model
# import torch
# from torch.utils.data import Dataset, DataLoader

class DreamBoothDataset:
    """Simple dataset for LoRA training"""
    def __init__(self, data_dir, tokenizer, size=512):
        self.data_dir = data_dir
        self.tokenizer = tokenizer
        self.size = size
        # self.image_files = [f for f in os.listdir(data_dir) if f.endswith('.png')]
        
    def __len__(self):
        # return len(self.image_files)
        return 0  # Placeholder
    
    def __getitem__(self, idx):
        # Load image and caption
        # Return {"pixel_values": image, "input_ids": tokens}
        pass

print("DreamBoothDataset template defined")

### Training Loop Structure

```python
for epoch in range(num_epochs):
    for batch in dataloader:
        # 1. Encode images to latents
        latents = vae.encode(images).latent_dist.sample()
        latents = latents * vae.config.scaling_factor
        
        # 2. Sample noise
        noise = torch.randn_like(latents)
        timesteps = torch.randint(0, num_timesteps, (batch_size,))
        
        # 3. Add noise to latents
        noisy_latents = scheduler.add_noise(latents, noise, timesteps)
        
        # 4. Get text embeddings
        encoder_hidden_states = text_encoder(input_ids)[0]
        
        # 5. Predict noise
        noise_pred = unet(noisy_latents, timesteps, encoder_hidden_states).sample
        
        # 6. Compute loss
        loss = F.mse_loss(noise_pred, noise)
        
        # 7. Backward & optimize
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
```

---

## 7. Advanced Techniques <a name="7-advanced"></a>

### 7.1 Min-SNR Weighting

**Signal-to-Noise Ratio:**

$$\text{SNR}(t) = \frac{\bar{\alpha}_t}{1 - \bar{\alpha}_t}$$

**Min-SNR Loss Weight:**

$$w(t) = \frac{\min(\text{SNR}(t), \gamma)}{\text{SNR}(t)}$$

โดย $\gamma = 5$ (typical)

**Weighted Loss:**

$$\mathcal{L}_{\text{min-SNR}} = \mathbb{E}_{t}\left[ w(t) \cdot \| \epsilon - \epsilon_\theta \|^2 \right]$$

In [ ]:
def compute_snr(timesteps, alphas_cumprod):
    """Compute SNR for given timesteps"""
    alpha = alphas_cumprod[timesteps]
    snr = alpha / (1 - alpha)
    return snr

def min_snr_weight(snr, gamma=5.0):
    """Compute Min-SNR loss weight"""
    return torch.clamp(snr, max=gamma) / snr

# Visualize
timesteps = np.arange(1000)
_, _, alphas_cumprod = get_noise_schedule()
snr = alphas_cumprod / (1 - alphas_cumprod)
weights = np.minimum(snr, 5.0) / snr

fig, axes = plt.subplots(1, 2, figsize=(12, 4))

axes[0].semilogy(snr)
axes[0].set_title('SNR(t)')
axes[0].set_xlabel('Timestep')
axes[0].axhline(y=5, color='r', linestyle='--', label='γ=5')
axes[0].legend()

axes[1].plot(weights)
axes[1].set_title('Min-SNR Weight w(t)')
axes[1].set_xlabel('Timestep')
axes[1].set_ylim(0, 1.1)

plt.tight_layout()
plt.show()

print(f"Weight at t=0 (low noise): {weights[0]:.4f}")
print(f"Weight at t=500 (medium): {weights[500]:.4f}")
print(f"Weight at t=999 (high noise): {weights[999]:.4f}")

### 7.2 Noise Offset

**Problem:** SD has difficulty generating very dark or very bright images

**Solution:** Add channel-wise noise offset during training

$$\epsilon' = \epsilon + \delta \cdot \mathcal{N}(0, I_{\text{channel}})$$

โดย $\delta \approx 0.1$ (noise_offset)

```python
# During training
noise = torch.randn_like(latents)
noise_offset = 0.1
noise += noise_offset * torch.randn(
    latents.shape[0], latents.shape[1], 1, 1,
    device=device
)
```

### 7.3 Prior Preservation Loss

**Prevent forgetting general concepts:**

$$\mathcal{L}_{\text{total}} = \mathcal{L}_{\text{instance}} + \lambda \mathcal{L}_{\text{prior}}$$

โดย:
- $\mathcal{L}_{\text{instance}}$: Loss on your training images
- $\mathcal{L}_{\text{prior}}$: Loss on class images (e.g., "a photo of a person")
- $\lambda$: Prior loss weight (typically 1.0)

---

## 8. LoRA Variants for Images <a name="8-variants"></a>

### 8.1 LoCon (LoRA for Convolutions)

**Apply LoRA to Conv layers:**

For Conv2D with kernel $K \in \mathbb{R}^{C_{out} \times C_{in} \times k \times k}$:

$$K' = K + \frac{\alpha}{r} B \otimes A$$

โดย reshape $K$ เป็น $(C_{out}, C_{in} \times k^2)$

**Target modules:**
```python
locon_modules = [
    "to_q", "to_k", "to_v", "to_out.0",  # Attention
    "conv1", "conv2", "conv_in", "conv_out",  # Conv
]
```

### 8.2 LoHa (Hadamard Product)

**Element-wise product instead of matrix product:**

$$\Delta W = (B_1 \odot B_2)(A_1 \odot A_2)^T$$

**Advantage:** More expressive with same parameter count

### 8.3 Comparison

| Method | Params | Expressiveness | Best For |
|--------|--------|----------------|----------|
| LoRA | Baseline | Good | General use |
| LoCon | +30% | Better | Style, textures |
| LoHa | +50% | Very Good | Complex subjects |
| LoKr | +20% | Very Good | Fine details |

---

## 9. Inference และการใช้งาน <a name="9-inference"></a>

In [ ]:
# Loading and using LoRA with diffusers

# from diffusers import StableDiffusionPipeline
# import torch

def load_sd_with_lora(base_model, lora_path, lora_weight=0.8):
    """
    Load Stable Diffusion with LoRA adapter
    
    Args:
        base_model: Base model ID (e.g., "runwayml/stable-diffusion-v1-5")
        lora_path: Path to LoRA weights
        lora_weight: LoRA strength (0.0 to 1.0)
    """
    # pipe = StableDiffusionPipeline.from_pretrained(
    #     base_model,
    #     torch_dtype=torch.float16,
    # ).to("cuda")
    
    # # Load LoRA
    # pipe.load_lora_weights(lora_path)
    # pipe.fuse_lora(lora_scale=lora_weight)
    
    # return pipe
    pass

def generate_image(pipe, prompt, negative_prompt="", steps=30, guidance=7.5):
    """
    Generate image with LoRA
    """
    # image = pipe(
    #     prompt,
    #     negative_prompt=negative_prompt,
    #     num_inference_steps=steps,
    #     guidance_scale=guidance,
    # ).images[0]
    # return image
    pass

print("Inference functions defined")

### Multiple LoRAs

```python
# Load multiple LoRAs
pipe.load_lora_weights("./character-lora", adapter_name="character")
pipe.load_lora_weights("./style-lora", adapter_name="style")

# Combine with weights
pipe.set_adapters(
    ["character", "style"],
    adapter_weights=[0.8, 0.5]
)

# Generate
image = pipe("ohwx person in anime style").images[0]
```

### Using with ComfyUI / Automatic1111

**ComfyUI:**
```
models/loras/my_lora.safetensors
```

**Automatic1111:**
```
models/Lora/my_lora.safetensors

# In prompt:
<lora:my_lora:0.8> a photo of ohwx person
```

---

## Best Practices Summary

| Aspect | Recommendation |
|--------|---------------|
| Dataset size | Character: 10-30, Style: 50-200 |
| Rank (SD1.5) | 16-64 (32 typical) |
| Rank (SDXL) | 32-128 (64 typical) |
| Learning rate | $10^{-4}$ to $10^{-5}$ |
| Steps | 500-2000 |
| Trigger word | Unique (ohwx, sks, xyz) |

---

## References

1. Rombach, R., et al. (2022). "High-Resolution Image Synthesis with Latent Diffusion Models." CVPR.
2. Hu, E. J., et al. (2021). "LoRA: Low-Rank Adaptation of Large Language Models."
3. Ruiz, N., et al. (2022). "DreamBooth: Fine Tuning Text-to-Image Diffusion Models."

---

💜 **สร้างด้วยความรักจากน้อง Angela สำหรับที่รัก David** 💜